# Data cleansing

Importing packages

In [1]:
import pandas as pd
import numpy as np
import os
#### last one only needed for first function
import ctypes

Funtion sorting and listing data files in raw folder for easier access to file names 
(sends pop-up window if there is a not data type file in data folder)

In [2]:
def only_data_files(filepath):
    for filename in os.listdir(filepath):
        if filename[-4:]==".csv" or filename[-5:]==".xlsx":
            print(filename)
        else:
            try:
                ctypes.windll.user32.MessageBoxW(0, "You have an unnecessary file in the folder: "+filename, "Warning", 0)
            except:
                continue
only_data_files("C:/Users/Szilvi/Documents/projects/coding-assignment/data/raw")

euribor_3M.csv
Europe_3_Factors.csv
eurostoxx.xlsx
gov_bond_yield_10Y.csv
gov_bond_yield_2Y.csv
gov_bond_yield_3Y.csv
gov_bond_yield_5Y.csv
gov_bond_yield_7Y.csv
longt_ir_10Y.csv
msci_gross.xlsx
msci_net_price.xlsx
msci_net_price_rt.xlsx
msci_price.xlsx
msci_price_rt.xlsx


Reading and converting differnet data sets (Fama French factors, euribor, bonds, indexes)

In [3]:
data=pd.read_csv("C:/Users/Szilvi/Documents/projects/coding-assignment/data/raw/Europe_3_Factors.csv"
                 , delimiter=",", skiprows=7, names=["date", "market_pr", "SMB", "HML", "RF"]
                 , nrows=364, dtype={"date":str})

In [4]:
euribor=pd.read_csv("C:/Users/Szilvi/Documents/projects/coding-assignment/data/raw/euribor_3M.csv"
                    , delimiter=",", skiprows=5, names=["date", "euribor"], dtype={"date":str})
d={"Jan":"01", "Feb":"02", "Mar":"03", "Apr":"04", "May":"05", "Jun":"06", 
   "Jul":"07", "Aug":"08", "Sep": "09", "Oct":"10", "Nov":"11", "Dec":"12"}
euribor.date.replace(d, inplace=True, regex=True)

In [5]:
bonds={}
d={"Jan":"01", "Feb":"02", "Mar":"03", "Apr":"04", "May":"05", "Jun":"06", 
   "Jul":"07", "Aug":"08", "Sep": "09", "Oct":"10", "Nov":"11", "Dec":"12"}

for filename in ["gov_bond_yield_2Y", "gov_bond_yield_3Y", "gov_bond_yield_5Y", 
                 "gov_bond_yield_7Y", "gov_bond_yield_10Y", "longt_ir_10Y"]: 
    value=pd.read_csv("C:/Users/Szilvi/Documents/projects/coding-assignment/data/raw/"+filename+".csv", 
                      delimiter=",", skiprows=5, names=["date", filename])
    value.date.replace(d, inplace=True, regex=True)
    bonds[filename]=value

In [6]:
index={}
for filename in ["msci_gross", "msci_net_price", "msci_net_price_rt", "msci_price", "msci_price_rt", "eurostoxx"]: 
    value=pd.read_excel("C:/Users/Szilvi/Documents/projects/coding-assignment/data/raw/"+filename+".xlsx", 
                        skiprows=17, usecols=[0,1], names=["date", filename], dtype={"date":str})
    value.date=value.date.str[0:4]+value.date.str[5:7]
    value=value.groupby("date").mean()
    index[filename]=value

Merging data sets into final dataframe

In [7]:
final_data=pd.merge(data, euribor, on="date", how="outer")
for i in bonds:
    final_data=pd.merge(final_data, bonds[i], on="date", how="outer")
for i in index:
    final_data=pd.merge(final_data, index[i], on="date", how="outer")
final_data.sort_values(by="date", inplace=True)

Calculating five year return

In [8]:
yld=list(final_data["gov_bond_yield_5Y"])
price=[]
return_5Y=[0]
for i in range(len(yld)):
    price.append(100/(1+yld[i])**5)
for i in range(1,len(price)):
    return_5Y.append(price[i]/price[i-1]-1)

final_data["return_5Y"]=return_5Y

Saving derived data

In [9]:
final_data.fillna(0, inplace=True)
final_data.to_csv("C:/Users/Szilvi/Documents/projects/coding-assignment/data/derived/data.csv")